<a href="https://colab.research.google.com/github/eddieward21/transfer-learning/blob/main/InceptionV3_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import os
import urllib.request
import random
import zipfile
from shutil import copyfile


import tensorflow.keras
#ImageDataGenerator
#InceptionV3
#model
#layers
